In [ ]:
#pip install git+https://github.com/huggingface/diffusers.git transformers accelerate
import requests
import torch
from PIL import Image
from io import BytesIO

from diffusers import StableUnCLIPImg2ImgPipeline

In [ ]:
#Start the StableUnCLIP Image variations pipeline
pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-unclip", torch_dtype=torch.float16, variation="fp16"
)
pipe = pipe.to("cuda")

In [ ]:
import numpy as np
# Load the train_images NumPy array
train_images = np.load("/mnt/sphere/projects/simon/Visual-Reconstruction/data/thingseeg2_metadata/train_images.npy", mmap_mode='r')
# Convert each image to a PIL image
pil_images = [Image.fromarray(image).convert("RGB") for image in train_images]
init_image = pil_images[1]
# Example: Display the first image
import matplotlib.pyplot as plt
# Plot the init_image
plt.imshow(init_image)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
init_image = Image.open('/mnt/sphere/projects/simon/Visual-Reconstruction/data/thingseeg2_metadata/test_images_direct/0.png').convert("RGB")

import matplotlib.pyplot as plt
# Plot the init_image
plt.imshow(init_image)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
device = pipe._execution_device
noise_level = torch.tensor([0], device=device)
embedding = pipe._encode_image(init_image, device=device, batch_size=1, num_images_per_prompt=1, do_classifier_free_guidance=True,noise_level=noise_level,generator=None, image_embeds = None)

In [ ]:
embedding = embedding[1].unsqueeze(0)
negative_prompt_embeds = torch.zeros_like(embedding)
embedding = torch.cat([negative_prompt_embeds, embedding])
embedding.shape

In [ ]:
plt.plot(embedding[1].detach().cpu().numpy())

In [ ]:
embedding[1][1025:1025+512].sum()

In [ ]:
torch.manual_seed(0)
images = pipe.decode(embedding).images
import matplotlib.pyplot as plt
# Plot the init_image
plt.imshow(images[0])
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
#Get image from URL
url = "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/stable_unclip/tarsila_do_amaral.png"
response = requests.get(url)
init_image = Image.open(BytesIO(response.content)).convert("RGB")

import matplotlib.pyplot as plt
# Plot the init_image
plt.imshow(init_image)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:

#Pipe to make the variation
torch.manual_seed(0)
images = pipe(init_image).images
# images[0].save("tarsila_variation.png")


In [ ]:
import matplotlib.pyplot as plt
# Plot the init_image
plt.imshow(images[0])
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
device = pipe._execution_device
noise_level = torch.tensor([0], device=device)
embedding = pipe._encode_image(init_image, device=device, batch_size=1, num_images_per_prompt=1, do_classifier_free_guidance=True,noise_level=noise_level,generator=None, image_embeds = None)

In [ ]:
embedding.shape

In [ ]:
# Preserve the corresponding axis
embedding = embedding[1].unsqueeze(0)
embedding.shape

In [ ]:
# extra_portion = embedding[0,1024:]
extra_portion = torch.cat([torch.ones(512), torch.zeros(512)])

embedding = embedding[0,:1024]
embedding.shape

In [ ]:
plt.plot(extra_portion.detach().cpu().numpy())
extra_portion.detach().cpu().numpy().sum(), extra_portion[511]

In [ ]:
embeddings = np.load("/mnt/sphere/projects/simon/Visual-Reconstruction/cache/thingseeg2_preproc/predicted_embeddings/sub-01/regress_clipvision1024.npy", mmap_mode='r')
embedding = torch.tensor(embeddings[32], device=device, dtype=torch.float16)
embedding.shape

In [ ]:
torch_ones = torch.ones(512, dtype=torch.float16, device=device)
torch_zeros = torch.zeros(512, dtype=torch.float16, device=device)
extra_portion = torch.cat([torch_ones, torch_zeros])
embedding = torch.cat([embedding, extra_portion]).unsqueeze(0)
embedding.shape

In [ ]:
negative_prompt_embeds = torch.zeros_like(embedding)
embedding = torch.cat([negative_prompt_embeds, embedding])
embedding.shape

In [ ]:
# embedding[1]

In [ ]:
torch.manual_seed(0)
images = pipe.decode(embedding).images

In [ ]:
import matplotlib.pyplot as plt
# Plot the init_image
plt.imshow(images[0])
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
images